<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/Bee_Database_Write.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2020-04-17 Erik Samsoe

* Link to source
* Identify file type
* Convert file
* Remove commas
* Load into BigQuery

In [0]:
import pandas as pd

* "MPG_textport.txt" shared by H through "Box".  
* Marirose shared the link in Slack with Erik.
* Original File is hosted on Drive here: [MPG_textport.txt](https://drive.google.com/open?id=1-4VDXNMHJpgoB_BZENVhF3BaGJyYBBoI)
* This file is loaded into a text editor.  VIM in this case.  The .txt file appears to be in tsv format.
  * Remove commas with search and replace
  * Replace tabs with commas
* Upload file to Notebook (because of large size)
* Check file type

## Mount Google Drive (Alterate Source Data Load)


In [0]:
# this will mount authenticated user google drive folders
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
src_drive = '/content/drive/My Drive/MPG Ranch Datasets/Bees/Source/MPG_textport_nocommas.csv'

In [0]:
df = pd.read_csv(src_drive, encoding='utf-16')

In [31]:
# DtypeWarning: Columns (1,35,52,57)
df.iloc[:, [1,35,52,57]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75205 entries, 0 to 75204
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   OtherCATALOG      1289 non-null   object
 1   endMon            626 non-null    object
 2   Eventcode         48335 non-null  object
 3   ProjectDATA_Flag  76 non-null     object
dtypes: object(4)
memory usage: 2.3+ MB


In [26]:
df.head(2)

,BarcodeID,OtherCATALOG,Questionable_recordDATA,NestingDATA_Avalible,MicrosatDATA_Avalible,#ofmeasurements,#ofPHOTOS,Floralspec_INDEX,#of_Determinations,Specimen_status,modifier,typeSTATUS,Gynandromorph,Stylops,Mites,M,F,Queen,Nomen_key,GenusName,Subgenus,Species,Subspecies,AuthorDATE,MappedGenus,Mapped_subgenus,MappedSpeceis,MappedBEE,determiner,JD_date,date,StartDay,startMon,Year0,endDay,endMon,Year1,starttime,endtime,DecimalLat,DecimalLon,Location Description,Location_Name,GAP_mapped,ECO_mapped,ECOEPA_name,GRID_link_B,Level0_name,Level1_name,Level2_Name,Elev_meters,Radius of collection in meters,Eventcode,CollMeth,CollMeth_modifier,collectors,ProjectName,ProjectDATA_Flag,ProjectID,Floral_association,bbsl_oldcode,code,Conditions,barmetricstart,barmetricend,windspeedstart,windspeedend,tempstart,tempend,noteblock,datecreated
0,BBSL1000000,NaN,0,no,no,NaN,NaN,1.0,NaN,Pinned specimen,none,None,False,False,False,0,1,False,valid,Lasioglossum,(Evylaeus),cooleyi,NaN,(Crawford 1906),Lasioglossum,(Evylaeus),cooleyi,1.0,S.J. Burrows 2016,124,4 May 2015,4.0,May,2015.0,NaN,NaN,NaN,8:25:00 AM,3:51:00 PM,46.6812,-114.03677,MPG Ranch: Plot 240,MPG240,NaN,Montana Valley and Foothill grasslands ...,Middle Rockies ...,1533229.0,USA,Montana,Missoula,983.0,NaN,MPG-032,pt,Zero out of 21,E. Israel,MPG RANCH,NaN,101,No Floral Record,Logan,BBSL,Partly Cloudy (>50% sun),NaN,NaN,NaN,NaN,NaN,NaN,Notes-::Question notes-; mapped juristictions>...,2016-01-10 22:49:43.843000000
1,BBSL1000001,NaN,0,no,no,NaN,NaN,1.0,NaN,Pinned specimen,none,None,False,False,False,0,1,False,valid,Lasioglossum,(Evylaeus),cooleyi,NaN,(Crawford 1906),Lasioglossum,(Evylaeus),cooleyi,1.0,S.J. Burrows 2016,124,4 May 2015,4.0,May,2015.0,NaN,NaN,NaN,8:25:00 AM,3:51:00 PM,46.6812,-114.03677,MPG Ranch: Plot 240,MPG240,NaN,Montana Valley and Foothill grasslands ...,Middle Rockies ...,1533229.0,USA,Montana,Missoula,983.0,NaN,MPG-032,pt,Zero out of 21,E. Israel,MPG RANCH,NaN,101,No Floral Record,Logan,BBSL,Partly Cloudy (>50% sun),NaN,NaN,NaN,NaN,NaN,NaN,Notes-::Question notes-; mapped juristictions>...,2016-01-10 22:49:43.850000000


In [27]:
df.columns

Index(['BarcodeID', 'OtherCATALOG', 'Questionable_recordDATA',
       'NestingDATA_Avalible', 'MicrosatDATA_Avalible', '#ofmeasurements',
       '#ofPHOTOS', 'Floralspec_INDEX', '#of_Determinations',
       'Specimen_status', 'modifier', 'typeSTATUS', 'Gynandromorph', 'Stylops',
       'Mites', 'M', 'F', 'Queen', 'Nomen_key', 'GenusName', 'Subgenus',
       'Species', 'Subspecies', 'AuthorDATE', 'MappedGenus', 'Mapped_subgenus',
       'MappedSpeceis', 'MappedBEE', 'determiner', 'JD_date', 'date',
       'StartDay', 'startMon', 'Year0', 'endDay', 'endMon', 'Year1',
       'starttime', 'endtime', 'DecimalLat', 'DecimalLon',
       'Location Description', 'Location_Name', 'GAP_mapped', 'ECO_mapped',
       'ECOEPA_name', 'GRID_link_B', 'Level0_name', 'Level1_name',
       'Level2_Name', 'Elev_meters', 'Radius of collection in meters',
       'Eventcode', 'CollMeth', 'CollMeth_modifier', 'collectors',
       'ProjectName', 'ProjectDATA_Flag', 'ProjectID', 'Floral_association',
       'bbs

In [0]:
df = df[['BarcodeID', 'OtherCATALOG', 'Questionable_recordDATA',
       'NestingDATA_Avalible', 'MicrosatDATA_Avalible', '#ofmeasurements',
       '#ofPHOTOS', 'Floralspec_INDEX', '#of_Determinations',
       'Specimen_status', 'modifier', 'typeSTATUS', 'Gynandromorph', 'Stylops',
       'Mites', 'M', 'F', 'Queen', 'Nomen_key', 'GenusName', 'Subgenus',
       'Species', 'Subspecies', 'AuthorDATE', 'MappedGenus', 'Mapped_subgenus',
       'MappedSpeceis', 'MappedBEE', 'determiner', 'JD_date', 'date',
       'StartDay', 'startMon', 'Year0', 'endDay', 'endMon', 'Year1',
       'starttime', 'endtime', 'DecimalLat', 'DecimalLon',
       'Location Description', 'Location_Name', 'GAP_mapped', 'ECO_mapped',
       'ECOEPA_name', 'GRID_link_B', 'Level0_name', 'Level1_name',
       'Level2_Name', 'Elev_meters', 'Radius of collection in meters',
       'Eventcode', 'CollMeth', 'CollMeth_modifier', 'collectors',
       'ProjectName', 'ProjectDATA_Flag', 'ProjectID', 'Floral_association',
       'bbsl_oldcode', 'code', 'Conditions', 'barmetricstart', 'barmetricend',
       'windspeedstart', 'windspeedend', 'tempstart', 'tempend', 'noteblock',
       'datecreated']]

### Rename columns 
* to comply with BigQuery naming conventions

In [0]:
df.columns = ['BarcodeID', 'OtherCATALOG', 'Questionable_recordDATA',
       'NestingDATA_Avalible', 'MicrosatDATA_Avalible', 'num_ofmeasurements',
       'num_ofPHOTOS', 'Floralspec_INDEX', 'num_of_Determinations',
       'Specimen_status', 'modifier', 'typeSTATUS', 'Gynandromorph', 'Stylops',
       'Mites', 'M', 'F', 'Queen', 'Nomen_key', 'GenusName', 'Subgenus',
       'Species', 'Subspecies', 'AuthorDATE', 'MappedGenus', 'Mapped_subgenus',
       'MappedSpeceis', 'MappedBEE', 'determiner', 'JD_date', 'date',
       'StartDay', 'startMon', 'Year0', 'endDay', 'endMon', 'Year1',
       'starttime', 'endtime', 'DecimalLat', 'DecimalLon',
       'Location_Description', 'Location_Name', 'GAP_mapped', 'ECO_mapped',
       'ECOEPA_name', 'GRID_link_B', 'Level0_name', 'Level1_name',
       'Level2_Name', 'Elev_meters', 'Radius_of_collection_in_meters',
       'Eventcode', 'CollMeth', 'CollMeth_modifier', 'collectors',
       'ProjectName', 'ProjectDATA_Flag', 'ProjectID', 'Floral_association',
       'bbsl_oldcode', 'code', 'Conditions', 'barmetricstart', 'barmetricend',
       'windspeedstart', 'windspeedend', 'tempstart', 'tempend', 'noteblock',
       'datecreated']

### Set datatypes

In [0]:
# ?

# Push to BigQuery

In [0]:
table_name = '20_04_17_dev'

In [0]:
df.to_gbq('bee_biology_systematics_lab.' + table_name, 'mpg-bees', if_exists='replace')